In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bugFixLogic import bugFix

In [3]:
# import bugFixLogic_indexer as bugFix_indexer

In [ ]:
bugFix_indexer.bugFix()

In [3]:
bugFix()

[[[]], [[1, 2, 6, 72, 7, 33, 4]], [[3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7, 9, 3]], [[5, 4, 3, 2, 1]], [[5, 4, 3, 1, 2]]]
[[[]], [[1, 2, 4, 6, 7, 33, 72]], [[1, 1, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, 9, 9]], [[1, 2, 3, 4, 5]], [[1, 2, 3, 4, 5]]]
The folder 'FauxPyReport_SearchBasedBugFixing_sbfl_statement_2024_05_27_12_07_36_327068' has been deleted.
File '10.txt' copied from 'testcases/BuggyPrograms' to 'testcases/GeneratedTests'.
File 'test.py' created in 'testcases/GeneratedTests' with the converted Python code.
returned from fault localization

0
1
2
3
************************************************************
************************************************************
2500
def mergesort(arr):

    def merge(left, right):
        result = []
        result = 0
        j = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
  

# Bug Prediction

In [1]:
import numpy as np
import faiss
# Load the index
index = faiss.read_index('../flatIP_index_all.index')

In [2]:
import torch
from unixcoder import UniXcoder
import numpy as np
DEVICE = torch.device("cpu")
model = UniXcoder("microsoft/unixcoder-base")
model.to(DEVICE)

#returns numpy array of embeddings
def get_embeddings(model,tokens,device=DEVICE):
  tokens_ids = model.tokenize([tokens],max_length=512,mode="<encoder-only>")
  source_ids = torch.tensor(tokens_ids).to(device)
  _,embeddings = model(source_ids)
  normEmb =  torch.nn.functional.normalize(embeddings, dim=1)
  return normEmb.detach()

In [3]:
from torch.utils.data import DataLoader

def batch_tokenize(token_list, max_length=512, mode="<encoder-only>"):
    # Tokenize multiple tokens in batches
    token_ids = model.tokenize(token_list, max_length=max_length, mode=mode)
    return torch.tensor(token_ids)

def fitness_bug_code(tokens_list, device=DEVICE, batch_size=64):
    dataloader = DataLoader(tokens_list, batch_size=batch_size)
    distances = []
    for batch_tokens in dataloader:
        token_ids = batch_tokenize(batch_tokens).to(device)
        _, embeddings = model(token_ids)
        distances.append(embeddings.detach().cpu())
    distances = torch.cat(distances)
    D, I = index.search(distances, k=1)
    return D

In [5]:
code = """# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

Run inference on images, videos, directories, streams, etc.

Usage:
    $ python path/to/detect.py --source path/to/img.jpg --weights yolov5s.pt --img 640

import argparse
import os
import platform
import sys
from pathlib import Path

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.experimental import attempt_load
from utils.datasets import LoadImages, LoadStreams
from utils.general import (LOGGER, apply_classifier, check_img_size, check_imshow, check_requirements, check_suffix,
                           colorstr, increment_path, non_max_suppression, print_args, save_one_box, scale_coords,
                           strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors
from utils.torch_utils import load_classifier, select_device, time_sync


@torch.no_grad()
def run(weights=ROOT / 'yolov5s.pt',  # model.pt path(s)
        source=ROOT / 'data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    w = str(weights[0] if isinstance(weights, list) else weights)
    classify, suffix, suffixes = False, Path(w).suffix.lower(), ['.pt', '.onnx', '.tflite', '.pb', '']
    check_suffix(w, suffixes)  # check weights have acceptable suffix
    pt, onnx, tflite, pb, saved_model = (suffix == x for x in suffixes)  # backend booleans
    stride, names = 64, [f'class{i}' for i in range(1000)]  # assign defaults
    if pt:
        model = torch.jit.load(w) if 'torchscript' in w else attempt_load(weights, map_location=device)
        stride = int(model.stride.max())  # model stride
        names = model.module.names if hasattr(model, 'module') else model.names  # get class names
        if half:
            model.half()  # to FP16
        if classify:  # second-stage classifier
            modelc = load_classifier(name='resnet50', n=2)  # initialize
            modelc.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()
    elif onnx:
        if dnn:
            check_requirements(('opencv-python>=4.5.4',))
            net = cv2.dnn.readNetFromONNX(w)
        else:
            check_requirements(('onnx', 'onnxruntime-gpu' if torch.has_cuda else 'onnxruntime'))
            import onnxruntime
            session = onnxruntime.InferenceSession(w, None)
    else:  # TensorFlow models
        import tensorflow as tf
        if pb:  # https://www.tensorflow.org/guide/migrate#a_graphpb_or_graphpbtxt
            def wrap_frozen_graph(gd, inputs, outputs):
                x = tf.compat.v1.wrap_function(lambda: tf.compat.v1.import_graph_def(gd, name=""), [])  # wrapped import
                return x.prune(tf.nest.map_structure(x.graph.as_graph_element, inputs),
                               tf.nest.map_structure(x.graph.as_graph_element, outputs))

            graph_def = tf.Graph().as_graph_def()
            graph_def.ParseFromString(open(w, 'rb').read())
            frozen_func = wrap_frozen_graph(gd=graph_def, inputs="x:0", outputs="Identity:0")
        elif saved_model:
            model = tf.keras.models.load_model(w)
        elif tflite:
            if "edgetpu" in w:  # https://www.tensorflow.org/lite/guide/python#install_tensorflow_lite_for_python
                import tflite_runtime.interpreter as tflri
                delegate = {'Linux': 'libedgetpu.so.1',  # install libedgetpu https://coral.ai/software/#edgetpu-runtime
                            'Darwin': 'libedgetpu.1.dylib',
                            'Windows': 'edgetpu.dll'}[platform.system()]
                interpreter = tflri.Interpreter(model_path=w, experimental_delegates=[tflri.load_delegate(delegate)])
            else:
                interpreter = tf.lite.Interpreter(model_path=w)  # load TFLite model
            interpreter.allocate_tensors()  # allocate
            input_details = interpreter.get_input_details()  # inputs
            output_details = interpreter.get_output_details()  # outputs
            int8 = input_details[0]['dtype'] == np.uint8  # is TFLite quantized uint8 model
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    if pt and device.type != 'cpu':
        model(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model.parameters())))  # run once
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, img, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        if onnx:
            img = img.astype('float32')
        else:
            img = torch.from_numpy(img).to(device)
            img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255  # 0 - 255 to 0.0 - 1.0
        if len(img.shape) == 3:
            img = img[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        if pt:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(img, augment=augment, visualize=visualize)[0]
        elif onnx:
            if dnn:
                net.setInput(img)
                pred = torch.tensor(net.forward())
            else:
                pred = torch.tensor(session.run([session.get_outputs()[0].name], {session.get_inputs()[0].name: img}))
        else:  # tensorflow model (tflite, pb, saved_model)
            imn = img.permute(0, 2, 3, 1).cpu().numpy()  # image in numpy
            if pb:
                pred = frozen_func(x=tf.constant(imn)).numpy()
            elif saved_model:
                pred = model(imn, training=False).numpy()
            elif tflite:
                if int8:
                    scale, zero_point = input_details[0]['quantization']
                    imn = (imn / scale + zero_point).astype(np.uint8)  # de-scale
                interpreter.set_tensor(input_details[0]['index'], imn)
                interpreter.invoke()
                pred = interpreter.get_tensor(output_details[0]['index'])
                if int8:
                    scale, zero_point = output_details[0]['quantization']
                    pred = (pred.astype(np.float32) - zero_point) * scale  # re-scale
            pred[..., 0] *= imgsz[1]  # x
            pred[..., 1] *= imgsz[0]  # y
            pred[..., 2] *= imgsz[1]  # w
            pred[..., 3] *= imgsz[0]  # h
            pred = torch.tensor(pred)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Print time (inference-only)
            LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')

            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model path(s)')
    parser.add_argument('--source', type=str, default=ROOT / 'data/images', help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(FILE.stem, opt)
    return opt


def main(opt):
    check_requirements(exclude=('tensorboard', 'thop'))
    run(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)

--------------------------------------------------"""

In [10]:
code = """
x = 2
x = 2
x = 2
x = 2
x = 2
x = 2
x = 2
x = 2
x = 2

from django.core import exceptions
from django.utils.functional import cached_property

from . import BLANK_CHOICE_DASH
from .mixins import FieldCacheMixin


class ForeignObjectRel(FieldCacheMixin):
    x = 2
    x = 2
    x = 2
    x = 2
    x = 2
    x = 2

    # Field flags
    auto_created = True
    concrete = False
    editable = False
    is_relation = True

    # Reverse relations are always nullable (Django can't enforce that a
    # foreign key on the related model points to this model).
    null = True

    def __init__(self, field, to, related_name=None, related_query_name=None,
                 limit_choices_to=None, parent_link=False, on_delete=None):
        self.field = field
        self.model = to
        self.related_name = related_name
        self.related_query_name = related_query_name
        self.limit_choices_to = {} if limit_choices_to is None else limit_choices_to
        self.parent_link = parent_link
        self.on_delete = on_delete

        self.symmetrical = False
        self.multiple = True

    # Some of the following cached_properties can't be initialized in
    # __init__ as the field doesn't have its model yet. Calling these methods
    # before field.contribute_to_class() has been called will result in
    # AttributeError
    @cached_property
    def hidden(self):
        return self.is_hidden()

    @cached_property
    def name(self):
        return self.field.related_query_name()

    @property
    def remote_field(self):
        return self.field

    @property
    def target_field(self):
        x = 2
        x = 2
        x = 2
        target_fields = self.get_path_info()[-1].target_fields
        if len(target_fields) > 1:
            raise exceptions.FieldError("Can't use target_field for multicolumn relations.")
        return target_fields[0]

    @cached_property
    def related_model(self):
        if not self.field.model:
            raise AttributeError(
                "This property can't be accessed before self.field.contribute_to_class has been called.")
        return self.field.model

    @cached_property
    def many_to_many(self):
        return self.field.many_to_many

    @cached_property
    def many_to_one(self):
        return self.field.one_to_many

    @cached_property
    def one_to_many(self):
        return self.field.many_to_one

    @cached_property
    def one_to_one(self):
        return self.field.one_to_one

    def get_lookup(self, lookup_name):
        return self.field.get_lookup(lookup_name)

    def get_internal_type(self):
        return self.field.get_internal_type()

    @property
    def db_type(self):
        return self.field.db_type

    def __repr__(self):
        return '<%s: %s.%s>' % (
            type(self).__name__,
            self.related_model._meta.app_label,
            self.related_model._meta.model_name,
        )

    def get_choices(self, include_blank=True, blank_choice=BLANK_CHOICE_DASH, ordering=()):
        x = 2
        x = 2
        x = 2
        qs = self.related_model._default_manager.all()
        if ordering:
            qs = qs.order_by(*ordering)
        return (blank_choice if include_blank else []) + [
            (x.pk, str(x)) for x in qs
        ]

    def is_hidden(self):
        return bool(self.related_name) and self.related_name[-1] == '+'

    def get_joining_columns(self):
        return self.field.get_reverse_joining_columns()

    def get_extra_restriction(self, where_class, alias, related_alias):
        return self.field.get_extra_restriction(where_class, related_alias, alias)

    def set_field_name(self):
        # By default foreign object doesn't relate to any remote field (for
        # example custom multicolumn joins currently have no remote field).
        self.field_name = None

    def get_accessor_name(self, model=None):
        # This method encapsulates the logic that decides what name to give an
        # accessor descriptor that retrieves related many-to-one or
        # many-to-many objects. It uses the lowercased object_name + "_set",
        # but this can be overridden with the "related_name" option. Due to
        # backwards compatibility ModelForms need to be able to provide an
        # alternate model. See BaseInlineFormSet.get_default_prefix().
        opts = model._meta if model else self.related_model._meta
        model = model or self.related_model
        if self.multiple:
            # If this is a symmetrical m2m relation on self, there is no reverse accessor.
            if self.symmetrical and model == self.model:
                return None
        if self.related_name:
            return self.related_name
        return opts.model_name + ('_set' if self.multiple else '')

    def get_path_info(self, filtered_relation=None):
        return self.field.get_reverse_path_info(filtered_relation)

    def get_cache_name(self):
        return self.get_accessor_name()


class ManyToOneRel(ForeignObjectRel):
    def __init__(self, field, to, field_name, related_name=None, related_query_name=None,
                 limit_choices_to=None, parent_link=False, on_delete=None):
        super().__init__(
            field, to,
            related_name=related_name,
            related_query_name=related_query_name,
            limit_choices_to=limit_choices_to,
            parent_link=parent_link,
            on_delete=on_delete,
        )

        self.field_name = field_name

    def __getstate__(self):
        state = self.__dict__.copy()
        state.pop('related_model', None)
        return state

    def get_related_field(self):
        field = self.model._meta.get_field(self.field_name)
        if not field.concrete:
            raise exceptions.FieldDoesNotExist("No related field named '%s'" % self.field_name)
        return field

    def set_field_name(self):
        self.field_name = self.field_name or self.model._meta.pk.name


class OneToOneRel(ManyToOneRel):

    def __init__(self, field, to, field_name, related_name=None, related_query_name=None,
                 limit_choices_to=None, parent_link=False, on_delete=None):
        super().__init__(
            field, to, field_name,
            related_name=related_name,
            related_query_name=related_query_name,
            limit_choices_to=limit_choices_to,
            parent_link=parent_link,
            on_delete=on_delete,
        )

        self.multiple = False


class ManyToManyRel(ForeignObjectRel):

    def __init__(self, field, to, related_name=None, related_query_name=None,
                 limit_choices_to=None, symmetrical=True, through=None,
                 through_fields=None, db_constraint=True):
        super().__init__(
            field, to,
            related_name=related_name,
            related_query_name=related_query_name,
            limit_choices_to=limit_choices_to,
        )

        if through and not db_constraint:
            raise ValueError("Can't supply a through model and db_constraint=False")
        self.through = through

        if through_fields and not through:
            raise ValueError("Cannot specify through_fields without a through model")
        self.through_fields = through_fields

        self.symmetrical = symmetrical
        self.db_constraint = db_constraint

    def get_related_field(self):
        opts = self.through._meta
        if self.through_fields:
            field = opts.get_field(self.through_fields[0])
        else:
            for field in opts.fields:
                rel = getattr(field, 'remote_field', None)
                if rel and rel.model == self.model:
                    break
        return field.foreign_related_fields[0]

"""

In [4]:
code = """
def find_max_in_list(lst):
    if not lst:
        return None

    max_val = lst[0]
    for i in range(1, len(lst) + 1):  # Bug: The loop goes out of the list bounds
        if lst[i] > max_val:
            max_val = lst[i]
    return max_val
"""

In [10]:
code ="""
import os
import sys
def return_list_1_to_10_except_5():
    lst = []
    for i in enumerate(1, 11):
        if i == 5:
            break        
        lst.append(i)
    return lst
"""

In [12]:
code = """
import imp

def load_module_from_file(module_name, file_path):
    try:
        with open(file_path, 'rb') as file:
            module = imp.load_source(module_name, file_path, file)  # Deprecated function
        return module
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except ImportError as e:
        print(f"Import error: {e}")
        return None
"""

In [13]:
codeLines = code.splitlines()

distances = []
for line in codeLines:
    emb = get_embeddings(model, line)
    D, I = index.search(emb, k=1)
    s = 0
    for d in D[0]:
        s += d
    distances.append(s)

for d_i, d in enumerate(distances):
    print(str(d_i + 1) + ":" + str(d) + ':' + codeLines[d_i] )


1:0.7532272338867188:
2:0.7845664620399475:import imp
3:0.7532272338867188:
4:0.7347317934036255:def load_module_from_file(module_name, file_path):
5:0.640394389629364:    try:
6:0.9241745471954346:        with open(file_path, 'rb') as file:
7:0.7002644538879395:            module = imp.load_source(module_name, file_path, file)  # Deprecated function
8:0.8518260717391968:        return module
9:0.8181358575820923:    except FileNotFoundError:
10:0.8292942047119141:        print(f"File not found: {file_path}")
11:0.8225617408752441:        return None
12:0.784103512763977:    except ImportError as e:
13:0.838379442691803:        print(f"Import error: {e}")
14:0.8225617408752441:        return None


In [76]:
import re

def split_on_newline_not_in_string(data):
    # Regex pattern to match quoted strings and new lines
    pattern = re.compile(r'(".*?"|[^"\n]*)(\n|$)')
    result = []
    current_line = []
    
    for match in pattern.finditer(data):
        text, newline = match.groups()
        current_line.append(text)
        
        if newline:
            result.append(''.join(current_line))
            current_line = []

    return result

data = 'Hello, world!\n"This is a string\nwith a newline."\nAnother line.\nAnd "yet\nanother" line.'

result = split_on_newline_not_in_string(data)
for line in result:
    print((line))


Hello, world!
This is a string

Another line.
yet


In [48]:
# Function to retrieve a single vector by index
def get_vector_by_index(index, idx):
    return index.reconstruct(idx)

# Function to retrieve multiple vectors by their indices
def get_vectors_by_indices(index, indices):
    return [index.reconstruct(idx) for idx in indices]

# Example indices to retrieve
indices_to_retrieve = [0, 10, 50]

# Retrieve embeddings
retrieved_vectors = get_vectors_by_indices(index, indices_to_retrieve)

# Print the retrieved embeddings
for i, vector in zip(indices_to_retrieve, retrieved_vectors):
    print(type(vector))
    print(f"Vector at index {i}:", vector)



<class 'numpy.ndarray'>
Vector at index 0: [ 2.07462215e+00 -3.16574407e+00 -9.87603188e-01 -1.48601580e+00
 -2.14675546e+00  2.22756243e+00  1.21427786e+00  1.35498017e-01
 -1.10037971e+00  7.30628967e-01  6.69160187e-01 -1.39932483e-01
 -1.03107500e+00  6.23861611e-01 -7.18971640e-02 -1.87470901e+00
 -6.86799586e-02 -2.79651791e-01 -3.11766207e-01 -1.08925426e+00
 -7.39683926e-01 -3.22964382e+00  1.24762499e+00  9.80725288e-02
  9.88353729e-01 -7.81754702e-02  5.06306052e-01 -2.85846770e-01
  1.49491057e-01 -2.04214901e-01 -4.11464119e+00 -2.12247229e+00
 -1.89803517e+00  1.33649790e+00  4.40624267e-01  3.42427343e-02
  4.03089046e-01  1.27846226e-01 -1.44535494e+00  4.33441430e-01
  2.10682821e+00 -7.37798154e-01 -2.09496692e-01  2.12294683e-01
 -3.18505228e-01 -2.12788630e+00  2.00101927e-01  1.66748857e+00
  4.76944417e-01 -1.41751900e-01 -2.41038370e+00 -1.04630363e+00
  1.63107061e+00 -1.22227800e+00 -2.08796430e+00 -1.39754164e+00
 -4.53256416e+00 -2.53819227e+00  2.24861026e+0

In [20]:
emb = get_embeddings(model, """from utils.general import (LOGGER, apply_classifier, check_img_size, check_imshow, check_requirements, check_suffix,
""")
D, I = index.search(emb, k=3)
s = 0
print(D)
for d in D[0]:
    s += d
print(s)

[[1642.795  1761.898  1766.5033]]
5171.1962890625


In [ ]:
D, I = index.search(emb, k=5)  # search the top 5 similar vectors

print(I)  # indices of the most similar vectors
print(D)  # distances to the most similar vectors

NameError: name 'index' is not defined